In [ ]:
cd ..

In [ ]:
# import datasets
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from tklearn import datasets
from tklearn.metrics import TextClassificationMetric
from tklearn.nn.trainer import Trainer
from tklearn.nn.evaluator import Evaluator
from tklearn.nn.callbacks import ProgbarLogger
from tklearn.config import config, config_scope

In [ ]:
config['emotion/Trainer'] = {
    'epochs': 2,
}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


def tokenize(texts):
    return tokenizer(texts["text"].tolist(), padding="max_length", truncation=True)

In [ ]:
train_dset = (
    datasets.load_dataset('hf', 'dair-ai/emotion', split="train")
    .map(tokenize, batched=True)
    .rename_column('label', 'labels')
    .remove_columns(["text"])
)

# train_dset = train_dset[:500].to_pylist()

In [ ]:
valid_dset = (
    datasets.load_dataset('hf', "dair-ai/emotion", split="validation")
    .map(tokenize, batched=True)
    .rename_column('label', 'labels')
    .remove_columns(["text"])
)

In [ ]:
len(train_dset), len(valid_dset)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=8,
)

In [ ]:
callbacks = [ProgbarLogger()]

with config_scope('emotion'):
    trainer = Trainer(model, callbacks=callbacks)

In [ ]:
metric = TextClassificationMetric(num_labels=8)

evaluator = Evaluator(valid_dset, metric=metric, postprocessor='argmax')

In [ ]:
trainer.fit(train_dset, evaluator=evaluator)

In [ ]:
test_dset = (
    datasets.load_dataset('hf', "dair-ai/emotion", split="test")
    .map(tokenize, batched=True)
    .rename_column('label', 'labels')
    .remove_columns(["text"])
)

evaluator = Evaluator(test_dset, metric=metric, postprocessor='argmax')

In [ ]:
evaluator.evaluate(trainer)